In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv
/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv


# Github connection 

In [3]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 8.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 13.7 MB/s eta 0:00:0000:01


In [4]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/LorenzoBozzoni/RecSys_Challenge_Polimi_2024.git

Cloning into 'RecSys_Challenge_Polimi_2024'...
remote: Enumerating objects: 181, done.
remote: Counting objects: 100% (181/181), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 181 (delta 37), reused 172 (delta 36), pack-reused 0 (from 0)
Receiving objects: 100% (181/181), 7.38 MiB | 18.90 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [5]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'RecSys_Challenge_Polimi_2024':
        repo = r
        print('Repository found')

Repository found


## Commit and push some specific file

In [15]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
        print('Commit and push completed successfully')
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())
        print(f'Exception {Exception} while trying to committing')

In [10]:
%cd /kaggle/working/RecSys_Challenge_Polimi_2024

/kaggle/working/RecSys_Challenge_Polimi_2024


In [17]:
upload_file('/kaggle/working/RecSys_Challenge_Polimi_2024/recsys-challenge-2024.ipynb', './recsys-challenge-2024.ipynb', 'adding github connection from kaggle')

Commit and push completed successfully


# Data exploration 
## URM
The URM contains a user for each row and an item for each column

In [ ]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")
URM_all_dataframe.head(10)

In [ ]:
print ("The number of interactions is {}".format(len(URM_all_dataframe)))

In [ ]:
userID_unique = URM_all_dataframe["user_id"].unique()
itemID_unique = URM_all_dataframe["item_id"].unique()
n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))

In [ ]:
mapped_id, original_id = pd.factorize(URM_all_dataframe["user_id"].unique())
user_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(URM_all_dataframe["item_id"].unique())
item_original_ID_to_index = pd.Series(mapped_id, index=original_id)

In [ ]:
original_item_ID = 292
print("New index for item {} is {}".format(original_item_ID, item_original_ID_to_index[original_item_ID]))
URM_all_dataframe.head(n=10)

In [ ]:
URM_all_dataframe["user_id"] = URM_all_dataframe["user_id"].map(user_original_ID_to_index)
URM_all_dataframe["item_id"] = URM_all_dataframe["item_id"].map(item_original_ID_to_index)

In [ ]:
URM_all_dataframe.head(n=10)

In [ ]:
userID_unique = URM_all_dataframe["user_id"].unique()
itemID_unique = URM_all_dataframe["item_id"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))


In [ ]:
import scipy.sparse as sps

URM_all = sps.coo_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)))

URM_all

In [ ]:
URM_all.tocsr()

In [ ]:
item_popularity = np.ediff1d(URM_all.tocsc().indptr)
item_popularity = np.sort(item_popularity)
item_popularity

In [ ]:
plt.plot(item_popularity, 'ro')
plt.ylabel('Num Interactions ')
plt.xlabel('Sorted Item')
plt.show()

In [ ]:
ten_percent = int(n_items/10)

print("Average per-item interactions over the whole dataset {:.2f}".
      format(item_popularity.mean()))

print("Average per-item interactions for the top 10% popular items {:.2f}".
      format(item_popularity[-ten_percent:].mean()))

print("Average per-item interactions for the least 10% popular items {:.2f}".
      format(item_popularity[:ten_percent].mean()))

print("Average per-item interactions for the median 10% popular items {:.2f}".
      format(item_popularity[int(n_items*0.45):int(n_items*0.55)].mean()))

In [ ]:
print("Number of items with zero interactions {}".
      format(np.sum(item_popularity==0)))

In [ ]:
user_activity = np.ediff1d(URM_all.tocsr().indptr)
user_activity = np.sort(user_activity)


plt.plot(user_activity, 'ro')
plt.ylabel('Num Interactions ')
plt.xlabel('Sorted User')
plt.show()

## ICM
The ICM has one item for each row and one feature for each column

In [ ]:
ICM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")
ICM_all_dataframe.head(10)

In [ ]:
featureID_unique = ICM_all_dataframe["feature_id"].unique()
itemID_unique_ICM = ICM_all_dataframe["item_id"].unique()

In [ ]:
n_features = len(featureID_unique)
n_items_ICM = len(itemID_unique_ICM)
n_interactions_ICM = len(ICM_all_dataframe)

print ("Number of items\t {}, Number of features\t {}".format(n_items_ICM, n_features))
print ("Max ID items\t {}, Max Id features\t {}\n".format(max(itemID_unique_ICM), max(featureID_unique)))

In [ ]:
mapped_id, original_id = pd.factorize(ICM_all_dataframe["feature_id"].unique())
feature_original_ID_to_index = pd.Series(mapped_id, index=original_id)

mapped_id, original_id = pd.factorize(ICM_all_dataframe["item_id"].unique())
item_original_ID_to_index_ICM = pd.Series(mapped_id, index=original_id)

In [ ]:
ICM_all_dataframe["feature_id"] = ICM_all_dataframe["feature_id"].map(feature_original_ID_to_index)
ICM_all_dataframe["item_id"] = ICM_all_dataframe["item_id"].map(item_original_ID_to_index_ICM)

In [ ]:
ICM_all_dataframe.head(n=10)

In [ ]:
featureID_unique = ICM_all_dataframe["feature_id"].unique()
itemID_unique_ICM = ICM_all_dataframe["item_id"].unique()

n_features = len(featureID_unique)
n_items_ICM = len(itemID_unique_ICM)
n_interactions_ICM = len(ICM_all_dataframe)

print ("Number of items\t {}, Number of features\t {}".format(n_items_ICM, n_features))
print ("Max ID items\t {}, Max Id features\t {}\n".format(max(itemID_unique_ICM), max(featureID_unique)))
print ("Average interactions per feature {:.2f}".format(n_interactions_ICM/n_features))
print ("Average interactions per item {:.2f}\n".format(n_interactions_ICM/n_items_ICM))

print ("Sparsity {:.2f} %".format((1-float(n_interactions_ICM)/(n_items_ICM*n_features))*100))


In [ ]:
ICM_all = sps.coo_matrix((ICM_all_dataframe["data"].values, (ICM_all_dataframe["item_id"].values,ICM_all_dataframe["feature_id"].values)))


ICM_all

In [ ]:
ICM_all.tocsr()

In [ ]:
item_popularity_ICM = np.ediff1d(ICM_all.tocsr().indptr)       # tocsc(), when rows and columns inverted
item_popularity_ICM = np.sort(item_popularity_ICM)
item_popularity_ICM 

In [ ]:
plt.plot(item_popularity_ICM, 'ro')
plt.ylabel('Num of features')
plt.xlabel('Sorted Item')
plt.show()

In [ ]:
ten_percent = int(n_items_ICM/10)

print("Average per-item interactions over the whole dataset {:.2f}".
      format(item_popularity_ICM.mean()))

print("Average per-item interactions for the top 10% popular items {:.2f}".
      format(item_popularity_ICM[-ten_percent:].mean()))

print("Average per-item interactions for the least 10% popular items {:.2f}".
      format(item_popularity_ICM[:ten_percent].mean()))

print("Average per-item interactions for the median 10% popular items {:.2f}".
      format(item_popularity_ICM[int(n_items_ICM*0.45):int(n_items_ICM*0.55)].mean()))

# Creating a validation set for testing our recommendation models

In [24]:
%cd /kaggle/working/RecSys_Challenge_Polimi_2024

/kaggle/working/RecSys_Challenge_Polimi_2024


In [25]:
from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout

dataset_reader = CustomDatasetReader(URM_path="/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv", ICM_path="/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")
dataSplitter = DataSplitter_Holdout(dataset_reader)
dataSplitter.load_data()
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()
ICM_train = dataset_reader.ICM

NameError: name 'CustomDatasetReader' is not defined